In [1]:
# Init
import os
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports
import numpy as np
import pandas as pd
import warnings
import multiprocessing
np.set_printoptions(suppress=True)
from numpy import concatenate as c
from numpy import cumsum
import random
%load_ext line_profiler

In [2]:
from generateSim import (generatePopulation, generateGenome)
from stats import stats
from regulation import regulation
# from checkCopyNumber import checkCopyNumber
from transposition import transposition
from recombination import recombination

In [3]:
from popSim import runBatch, runSim

In [4]:
gen,piset,piIndice, rates = generateGenome(
    numberOfInsertionSites=10000,
    numberOfChromosomes=6,
    baseRecombinationRate=0.1,
    baseSelection=0,
    baseTau=1,
)
pop, tr , TEset= generatePopulation(
    gen,
    piIndice,
    NumberOfIndividual=1000,
    NumberOfTransposonTypes=1,
    NumberOfInsertionsPerType=[1],
    FrequencyOfInsertions=[1.0],
    ExcisionRates = [1.0]
)

In [8]:
%lprun -f runSim runSim(gen,pop,tr,TEset,1,500,rates,piset)

Timer unit: 1e-06 s

Total time: 634.568 s
File: /home/siddharth/Documents/Projects/Simulicron/src/simulicronalpha/popSim.py
Function: runSim at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                           def runSim(
    27                                               genomeMatrix,
    28                                               populationMatrix,
    29                                               transposonMatrix,
    30                                               TEset,
    31                                               NumberOfTransposonInsertions,
    32                                               generations,
    33                                               genMap,
    34                                               piSet,
    35                                           ):
    36                                               # ------------------#
    37                                               # l

In [ ]:
runSim(gen,pop,tr,TEset,2,101,rates,piset)

In [ ]:
%lprun -f recombination recombination(rates=rates, transposonMatrix=tr, v1=[52], v2=[65])

In [ ]:
%lprun -f runBatch runBatch(numberOfSimulations=1,baseSelection=0,baseInsertionProb=1,numberOfInsertionSites=10000,numberOfChromosomes=6,baseRecombinationRate=0.01,baseTau=1,numberOfPiRNA=6,piPercentage=3,enablePiRecombination=False,NumberOfIndividual=1000,NumberOfTransposonTypes=1,NumberOfInsertionsPerType=[1],FrequencyOfInsertions=[1.0],ExcisionRates=[1.0],RepairRates=[1],InsertionRates=[1],HardyWeinberg=False,NumberOfGenerations=10,numberOfThreads=1,)

In [ ]:
np.set_printoptions(suppress=True,)

In [ ]:
%lprun -f transposition transposition(transposonMatrix=tr, genomeMatrix=gen, NumberOfTransposonInsertions=1, TEset=TEset, v1=[52,101,321,433,12,23,234,123,657,29], v2=[65,54,22,121,23])

In [ ]:
def checkCopyNumber(populationMatrix, TEset, transposonMatrix):
    # The local flatten lambda
    flatten = lambda *n: (
        e for a in n for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,))
    )
    # Convert the list to flat list and remove 0
    # if present.
    # Also create a dictionary counting 
    # each family type
    TEfamilyCount = {k: [] for k in range(1, len(TEset.keys()) + 1)}
    TEfamilyVar = {k: [] for k in range(1, len(TEset.keys()) + 1)}
    TETotal = []
    for i in range(len(populationMatrix)):
        v1 = populationMatrix[i, 0] if type(populationMatrix[i, 0]) is list else []
        v2 = populationMatrix[i, 1] if type(populationMatrix[i, 1]) is list else []
        if (v1 + v2 == []):
            TETotal.append(0)
            for k in TEset.keys():
                TEfamilyCount[k].append(0)
        else:
            # Check the occupancy for each family
            TEfamilies = transposonMatrix[v1+v2, 0].tolist()
            TETotal.append(len(TEfamilies))
            for k in TEset.keys():
                TEfamilyCount[k].append(TEfamilies.count(k))
    # Calculate variance per family and sum the count
    for k in TEset.keys():
        TEfamilyVar[k].append(np.var(TEfamilyCount[k]))
        TEfamilyCount[k] = sum(TEfamilyCount[k])
    return sum(TETotal), np.var(TETotal), TEfamilyCount, TEfamilyVar

In [ ]:
TEfamilyCount = {k: [] for k in range(1, len(TEset.keys()) + 1)}

In [ ]:
%%timeit
x, y, a, b = checkCopyNumber(pop, TEset, tr)

In [ ]:
b

In [ ]:
TEfamilyCount

In [ ]:
pop

In [ ]:
tr

In [ ]:
z = [3,4,6,74,4,3]

In [ ]:
z.count(7)